In [22]:
## all imports
import requests
from bs4 import BeautifulSoup
import time
import operator
import socket
#import cPickle
import re # regular expressions

import pandas as pd
from pandas import DataFrame


import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

#from secret import *

#This code is to import the libraries we will ptentially be using during the execution of the tasks I have been assigned to
# data analysis and wrangling
import pandas as pd
import pickle
import numpy as np
import random as rnd
from datetime import timedelta
import scipy
from scipy import stats
#Limiting floats output to 3 decimal points
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) 

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# machine learning and metrics
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score, make_scorer, mean_squared_error
from sklearn.model_selection import KFold,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, BayesianRidge, ElasticNet, Lasso, SGDRegressor, Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor,GradientBoostingRegressor,RandomForestRegressor
from sklearn.svm import SVC, LinearSVC, SVR, LinearSVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings('ignore')
import urllib
import urllib.request
import os
from string import ascii_lowercase
import csv

In [23]:
#from selenium import webdriver 
#from selenium.webdriver.common.by import By 
#from selenium.webdriver.support.ui import WebDriverWait 
#from selenium.webdriver.support import expected_conditions as EC 
#from selenium.common.exceptions import TimeoutException

In [24]:
FiveH_href=pd.read_csv('datasets/hrefs-players_final.csv')
playerlist=FiveH_href.iloc[:,0]
#playerlist.pop(253)


In [25]:
def createNewUrl(player):
    player_splitted=player.split("/")
    url="https://www.transfermarkt.co.uk/"+player_splitted[1]+"/leistungsdatendetails/spieler/"+player_splitted[4]
    return(url)

In [27]:
#url_detailed='https://www.transfermarkt.co.uk/luis-suarez/leistungsdatendetails/spieler/44352'

def scrape_url(url_player):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}
    try:
        response_detailed = requests.get(url, headers=headers, timeout=10)
        response_detailed.raise_for_status()

    except Exception:
        return None
    time.sleep(30)
    soup = BeautifulSoup(response_detailed.text,"html.parser")
    return(soup)

In [28]:
def wrangling_soup(soup):
    data = []
    table = soup.find('table', attrs={'class':'items'})
    table_body = table.find('tbody')

    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])
    return(data)



In [29]:
with open("players_stats_total.csv", "w") as f:
    writer = csv.writer(f,delimiter=',')
    for player in playerlist:
        url=createNewUrl(player)
        soup=scrape_url(url)
        data=wrangling_soup(soup)
        
        for i in data:
            i.insert(0,url)
            writer.writerow(i)
            

AttributeError: 'NoneType' object has no attribute 'find'

In [8]:
players_stats=pd.read_csv('players_stats_total.csv',  names=["names","season", "league", "apps","PPM","goals","assists", "yellow_1/yellow_2/red", "mins_played"] )

In [9]:
len(players_stats)

21708

In [33]:
players_stats["assists"][players_stats["names"]=="emiliano viviano"]=players_stats["yellow_1/yellow_2/red"]
players_stats["assists"][players_stats["names"]=="sebastien frey"]=players_stats["yellow_1/yellow_2/red"]
players_stats["assists"][players_stats["names"]=="eduardo"]=players_stats["yellow_1/yellow_2/red"]
players_stats["assists"][players_stats["names"]=="neto"]=players_stats["yellow_1/yellow_2/red"]
players_stats["assists"][players_stats["names"]=="maksym koval"]=players_stats["yellow_1/yellow_2/red"]
players_stats["assists"][players_stats["names"]=="federico marchetti"]=players_stats["yellow_1/yellow_2/red"]
players_stats["assists"][players_stats["names"]=="marco storari"]=players_stats["yellow_1/yellow_2/red"]
players_stats["assists"][players_stats["names"]=="roberto"]=players_stats["yellow_1/yellow_2/red"]
players_stats["assists"][players_stats["names"]=="ben foster"]=players_stats["yellow_1/yellow_2/red"]
players_stats["assists"][players_stats["names"]=="bernd leno"]=players_stats["yellow_1/yellow_2/red"]
players_stats["assists"][players_stats["names"]=="eduardo"]=players_stats["goals"]
players_stats["assists"][players_stats["names"]=="diego lopez"]=players_stats["yellow_1/yellow_2/red"]
players_stats["assists"][players_stats["names"]=="anders lindegaard"]=players_stats["yellow_1/yellow_2/red"]
players_stats["assists"][players_stats["names"]=="vladimir gabulov"]=players_stats["yellow_1/yellow_2/red"]
players_stats["assists"][players_stats["names"]=="ali al habsi"]=players_stats["yellow_1/yellow_2/red"]
players_stats["assists"][players_stats["names"]=="salvatore sirigu"]=players_stats["yellow_1/yellow_2/red"]
players_stats["assists"][players_stats["names"]=="shay given"]=players_stats["yellow_1/yellow_2/red"]
players_stats.iloc[6526]

names                        shay given
season                            16/17
league                   Premier League
apps                                  5
PPM                                0,20
goals                                 -
assists                              14
yellow_1/yellow_2/red                14
mins_played                           -
Name: 6526, dtype: object

In [34]:
#players_stats.drop(["league","PPM","yellow_1/yellow_2/red"], axis=1,inplace=True)
#for i in range(len(players_stats.season)):
    #players_stats.season.iloc[i]=("20"+players_stats.season.iloc[i][3:])

cols=["apps","goals","assists", "mins_played"]
for i in cols:
    players_stats[i]=players_stats[i].str.replace('-','', regex=True)
    players_stats[i]=players_stats[i].str.replace(" /  / ","", regex=True)
    players_stats[i]=players_stats[i].str.replace("/","", regex=True)
    players_stats[i]=players_stats[i].str.replace(" ","", regex=True)
    players_stats[i]=players_stats[i].str.replace("'","", regex=True)
    players_stats[i]=pd.to_numeric(players_stats[i])

In [52]:
players_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21130 entries, 0 to 21129
Data columns (total 6 columns):
names          21130 non-null object
season         19768 non-null datetime64[ns]
apps           20210 non-null float64
goals          9548 non-null float64
assists        8427 non-null float64
mins_played    20029 non-null float64
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 990.5+ KB


In [36]:
players_stats.drop(["league","PPM","yellow_1/yellow_2/red"], axis=1,inplace=True)

In [37]:
players_stats.season="20"+ players_stats.season.str[3:]
#for i in range(len(players_stats.season)):
    #players_stats.season.iloc[i]=("20"+players_stats.season.iloc[i][3:])
#players_stats.season.replace('/','',inplace=True)
players_stats.season

0        2019
1        2019
2        2018
3        2018
4        2018
5        2017
6        2017
7        2017
8        2016
9        2016
10       2015
11       2015
12       2015
13       2015
14       2015
15       2014
16       2014
17       2014
18       2014
19       2014
20       2013
21       2013
22       2013
23       2012
24       2012
25       2011
26       2011
27       2011
28       2010
29       2010
         ... 
21100    2014
21101    2014
21102    2014
21103    2013
21104    2013
21105    2013
21106    2012
21107    2012
21108    2011
21109    2011
21110    2011
21111    2011
21112    2010
21113    2010
21114    2010
21115    2010
21116    2009
21117    2009
21118    2009
21119    2009
21120    2008
21121    2008
21122    2007
21123    2007
21124    2006
21125    2005
21126    2005
21127    2004
21128    2004
21129    2003
Name: season, Length: 21130, dtype: object

In [51]:
players_stats['season']=pd.to_datetime(players_stats.season, errors='coerce')#.astype('datetime64')
players_stats.drop(['season1'],axis=1,inplace=True)
players_stats.to_csv("players_stats.csv")
